This script parses the Wilco weekly inventory report and creates a *clean* version of the report. The wilco convention for naming files seems to be newstock-{data}T{N}.xlsx. 
The output file name created is wilco-{data}.xlsx

In [42]:
import re
import os
import openpyxl
from pathlib import Path

IN_DIR = 'sample1'
OUT_DIR = Path(IN_DIR, 'out')
if not os.path.exists(OUT_DIR):
    print("Creating output directory")
    os.mkdir(OUT_DIR)

print(f"Storing all outputs to {OUT_DIR}")


Storing all outputs to sample1/out


In [43]:
IN_XLSX = Path(IN_DIR, "newstock - 2022-08-05T201013.974.xlsx")
m = re.search(r'newstock - (.*)T', str(IN_XLSX))
if m:
    out_fname = "wilco_" + m.group(1) + ".xlsx"
    OUT_XLSX = Path(OUT_DIR, out_fname)
else:
    raise ValueError("Input filename does not match naming convention.")
    
print(f"Parsing {IN_XLSX} to create output file {OUT_XLSX}")

Parsing sample1/newstock - 2022-08-05T201013.974.xlsx to create output file sample1/out/wilco_2022-08-05.xlsx


In [44]:
wb = openpyxl.load_workbook(IN_XLSX)
print(wb)

In [45]:
sheetnames = wb.sheetnames
print(sheetnames)

['newstock']


In [46]:
sheet = wb['newstock']

In [47]:
HEADER_COLUMN = ['SKU', 'LOCATION', 'Batch #', 'Description', 'OnHand', 'Allocated', 'Available']
header_row_idx = None

for row in sheet.iter_rows(max_row=10):
   col_values = [cell.value for cell in row]
   for i in range(len(HEADER_COLUMN)):
        if col_values[i] != HEADER_COLUMN[i]:
            break
        header_row_idx = row[0].row
        break

In [48]:
print(header_row_idx)

7


In [49]:
inventory = []
type_item = ""
for row in sheet.iter_rows(min_row=header_row_idx + 1, values_only=True):
    if 'Details' in row[0]:
        continue
        
    if row[0] == "":
        # Some rows are for totals
        continue
    inventory.append((row[0].replace('\n',''), row[6], row[9]))
    

In [50]:
outbook = openpyxl.Workbook()
outsheet = outbook.active
outsheet["A1"] = "SKU"
outsheet["B1"] = "Description"
outsheet["C1"] = "OnHand"

In [51]:
for row, (sku, description, onhand) in enumerate(inventory, start=2):
    outsheet[f"A{row}"] = sku
    outsheet[f"B{row}"] = description
    outsheet[f"C{row}"] = onhand
outbook.save(OUT_XLSX)